In [1]:
!pip install pfrl@git+https://github.com/voidful/pfrl.git
!pip install textrl==0.2.13

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/voidful/pfrl.git to /tmp/pip-install-0urhrjwh/pfrl_9acc2132d6a3476a9f2114be11b71fcb
  Running command git clone --filter=blob:none --quiet https://github.com/voidful/pfrl.git /tmp/pip-install-0urhrjwh/pfrl_9acc2132d6a3476a9f2114be11b71fcb
  Resolved https://github.com/voidful/pfrl.git to commit 2ad3d51a7a971f3fe7f2711f024be11642990d61
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from textrl import TextRLEnv,TextRLActor
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelWithLMHead
import logging
import sys
import pfrl
import torch
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


In [3]:
tokenizer = AutoTokenizer.from_pretrained("huggingtweets/elonmusk")  
model = AutoModelWithLMHead.from_pretrained("huggingtweets/elonmusk")
model.eval()
model.cuda()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1352: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [4]:
sentiment = pipeline('sentiment-analysis',model="cardiffnlp/twitter-roberta-base-sentiment",tokenizer="cardiffnlp/twitter-roberta-base-sentiment",device=0,return_all_scores=True)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/open_llama/modeling_open_llama.py:42: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn(
Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [5]:
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.CRITICAL)

In [6]:
sentiment("dogecoin is good")

[[{'label': 'LABEL_0', 'score': 0.0030372946057468653},
  {'label': 'LABEL_1', 'score': 0.03901847079396248},
  {'label': 'LABEL_2', 'score': 0.9579441547393799}]]

In [7]:
sentiment("dogecoin is bad")[0][0]['score']

0.9338533878326416

In [8]:
observaton_list = [['i think dogecoin is']]

In [9]:
class MyRLEnv(TextRLEnv):
    def get_reward(self, input_item, predicted_list, finish): # predicted will be the list of predicted token
      reward = 0
      if finish:
        predicted_text = tokenizer.convert_tokens_to_string(predicted_list[0])
        if sentiment(predicted_text)[0][0]['score'] > 0.5:
          reward = sentiment(predicted_text)[0][0]['score']*100
        else:
          reward = -sentiment(predicted_text)[0][2]['score']*100 -sentiment(predicted_text)[0][1]['score']*100
      return reward

In [10]:
env = MyRLEnv(model, tokenizer, observation_input=observaton_list,compare_sample=1)
actor = TextRLActor(env,model,tokenizer)
agent = actor.agent_ppo(update_interval=100, minibatch_size=3, epochs=300)

In [11]:
actor.predict(observaton_list[0])

[' a good idea<|endoftext|>']

In [12]:
pfrl.experiments.train_agent_with_evaluation(
    agent,
    env,
    steps=300,
    eval_n_steps=None,
    eval_n_episodes=1,       
    train_max_episode_len=100,  
    eval_interval=10,
    outdir='elon_musk_dogecoin', 
)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pfrl/agents/ppo.py:133: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  actions = torch.tensor([b["action"] for b in dataset], device=device)
/usr/local/lib/python3.10/dist-packages/textrl/actor.py:248: UserWarning: Using a target size (torch.Size([3, 1])) that is different to the input size (torch.Size([3, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss_value_func = F.mse_loss(vs_pred, vs_teacher)


(<textrl.actor.TextPPO at 0x7f5210d7f9d0>,
 [{'average_value': 4.771971,
   'average_entropy': 3.760759,
   'average_value_loss': nan,
   'average_policy_loss': nan,
   'n_updates': 0,
   'explained_variance': nan,
   'eval_score': -98.52138161659241},
  {'average_value': 4.688837,
   'average_entropy': 3.82466,
   'average_value_loss': nan,
   'average_policy_loss': nan,
   'n_updates': 0,
   'explained_variance': nan,
   'eval_score': -98.52138161659241},
  {'average_value': 4.739726,
   'average_entropy': 3.8794098,
   'average_value_loss': nan,
   'average_policy_loss': nan,
   'n_updates': 0,
   'explained_variance': nan,
   'eval_score': -98.52138161659241},
  {'average_value': 4.5619793,
   'average_entropy': 3.6960547,
   'average_value_loss': nan,
   'average_policy_loss': nan,
   'n_updates': 0,
   'explained_variance': nan,
   'eval_score': -98.52138161659241},
  {'average_value': 4.6455574,
   'average_entropy': 3.642983,
   'average_value_loss': nan,
   'average_policy_los

In [13]:
agent.load("/content/elon_musk_dogecoin/best")
actor.predict(observaton_list[0])

[' the best way to do it<|endoftext|>']

In [14]:
sentiment("i think dogecoin is the best way to do it")

[[{'label': 'LABEL_0', 'score': 0.40355002880096436},
  {'label': 'LABEL_1', 'score': 0.49569958448410034},
  {'label': 'LABEL_2', 'score': 0.10075046867132187}]]

In [15]:
!rm -r /content/elon_musk_dogecoin